# Generative AI: RAG & AI Agent
# LangChain Chains

## Simple Chain

In [14]:
!pip install -q langchain langchain-core


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
from langchain_aws import ChatBedrock
llm = ChatBedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    model_kwargs=dict(temperature=0),
    region="us-east-1"
)

In [5]:
from langchain.prompts import PromptTemplate

template = """
From the following Syslog message extract and explain:
1. Event type
2. Affected BGP peer
3. Probable cause
4. Recommended troubleshooting steps

Syslog message:
{message}
"""

prompt = PromptTemplate.from_template(template)


In [10]:
from langchain.schema import SystemMessage, HumanMessage

chain = prompt | llm

syslog_bgp = "*Aug  8 17:42:13.123: %BGP-5-ADJCHANGE: neighbor 192.0.2.1 Down BGP Notification sent"

messages = [
    SystemMessage(content="You are a senior network engineer analyzing Cisco IOS-XE syslog messages."),
    HumanMessage(content=prompt.format(message=syslog_bgp))
]

result = chain.invoke(messages)
print(result.content)

# Analysis of BGP Syslog Message

## 1. Event Type
This is a BGP adjacency change notification message (%BGP-5-ADJCHANGE) indicating that a BGP peering session has gone down. The severity level is 5, which is a notification-level message rather than a critical error.

## 2. Affected BGP Peer
The affected BGP peer is **192.0.2.1**. This is the IP address of the BGP neighbor with which the session has been terminated.

## 3. Probable Cause
The message indicates that a BGP Notification message was sent to the peer, causing the session to go down. BGP Notification messages are sent when a protocol error is detected. Common causes include:

- BGP hold timer expiration
- Malformed BGP messages
- BGP configuration changes
- Authentication failures
- Administrative shutdown
- Network connectivity issues between peers

Since the router sent the notification (rather than received it), the local router likely detected an issue with the peer or the connection.

## 4. Recommended Troubleshooting St

## Chain with JSON Parser

In [27]:
from langchain.output_parsers.json import SimpleJsonOutputParser
import json

parser = SimpleJsonOutputParser()

template = """
You are a senior network engineer.

Extract the following fields from the syslog message and return ONLY valid JSON:
- timestamp
- event_type
- affected_ip
- reason

Syslog message:
{message}

{format_instructions}
"""
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = llm | parser

messages = [
    SystemMessage(content="You are a senior network engineer analyzing Cisco IOS-XE syslog messages."),
    HumanMessage(content=prompt.format(message=syslog_bgp))
]

result = chain.invoke(messages)

print(json.dumps(result, indent=2))

{
  "timestamp": "Aug 8 17:42:13.123",
  "event_type": "BGP-5-ADJCHANGE",
  "affected_ip": "192.0.2.1",
  "reason": "Down BGP Notification sent"
}


## Multiple Chains

In [40]:
parser_syslog = SimpleJsonOutputParser()

syslog_prompt_template = """
You are a senior network engineer.

Extract the following fields from the syslog message and return ONLY valid JSON:
- timestamp
- event_type
- affected_ips (List of devices/IPs/node names)
- reason

Syslog message:
{message}

{format_instructions}
"""
syslog_prompt = PromptTemplate.from_template(
    syslog_prompt_template,
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

syslog_chain = syslog_prompt | llm | parser_syslog

itsm_prompt_template = """
You are an IT support engineer.

Create an ITSM incident ticket in JSON format with the following fields:
- title (short summary of the event)
- date: (Time the even took place)
- affected devices: (List of affected devices, or IPs)
- description (detailed context)
- troubleshooting_steps (array of step-by-step actions)
- severity
- assigned team
- status

Here is the parsed syslog data:
{syslog_json}
"""
parser_itsm = SimpleJsonOutputParser()

itsm_chain = itsm_prompt | llm | parser_itsm

# Combined chain: run syslog_chain → feed result into itsm_chain
combined_chain = {
    "syslog_json": syslog_chain  # this runs first
} | itsm_chain                   # output goes here

# Example syslog
syslog_bgp = "*Aug  8 17:13:02.456: %CRYPTO-4-IKMP_BAD_AUTH: IKE authentication between local address 198.51.100.10 and peer 203.0.113.5 failed. Possible pre-shared key"

ticket = combined_chain.invoke({"message": syslog_bgp})

print(json.dumps(ticket, indent=2))

{
  "title": "VPN Authentication Failure - IKE Pre-shared Key Issue",
  "date": "Aug 8 17:13:02.456",
  "description": "A VPN tunnel authentication failure was detected between IP addresses 198.51.100.10 and 203.0.113.5. The CRYPTO-4-IKMP_BAD_AUTH event indicates an IKE (Internet Key Exchange) authentication failure, likely due to mismatched pre-shared keys between the VPN endpoints. This could result in failed VPN connectivity between these systems.",
  "troubleshooting_steps": [
    "Verify the pre-shared key configuration on both VPN endpoints (198.51.100.10 and 203.0.113.5) to ensure they match exactly",
    "Check for any recent changes to VPN configurations on either device",
    "Examine the full crypto logs on both devices for additional context",
    "Verify that both devices are using compatible IKE versions and encryption settings",
    "Temporarily enable debug-level logging for IKE negotiations",
    "Test connectivity between the endpoints to ensure there are no underlyin